In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import RandomOverSampler

Загружаем данные. Students' performance
Текстовое описание набора данных: https://www.kaggle.com/datasets/nikhil7280/student-performance-multiple-linear-regression

In [45]:
student_performance = pd.DataFrame(pd.read_csv('Student_Performance.csv'))

Разведочный анализ данных. Узнаем, есть ли пустые значения, какие типы данных присутствуют

In [46]:
student_performance.isnull().sum()

Hours Studied                       0
Previous Scores                     0
Extracurricular Activities          0
Sleep Hours                         0
Sample Question Papers Practiced    0
Performance Index                   0
dtype: int64

In [47]:
student_performance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Hours Studied                     10000 non-null  int64  
 1   Previous Scores                   10000 non-null  int64  
 2   Extracurricular Activities        10000 non-null  object 
 3   Sleep Hours                       10000 non-null  int64  
 4   Sample Question Papers Practiced  10000 non-null  int64  
 5   Performance Index                 10000 non-null  float64
dtypes: float64(1), int64(4), object(1)
memory usage: 468.9+ KB


In [48]:
class_counts = student_performance["Performance Index"].value_counts()
print(class_counts)

Performance Index
67.0     187
45.0     185
56.0     181
49.0     181
40.0     180
        ... 
14.0       7
99.0       6
11.0       5
100.0      3
10.0       1
Name: count, Length: 91, dtype: int64


С помощью LabelEncoder представляем строковые данные в виде числовых (бинарные данные в столбце Extracurricular Activities)
1 == Yes, 0 == No

In [49]:
label_encoder = LabelEncoder()
student_performance['Extracurricular Activities_encoded'] = label_encoder.fit_transform(student_performance['Extracurricular Activities'])

In [50]:
student_performance = student_performance.drop('Extracurricular Activities', axis=1)

student_performance

,Hours Studied,Previous Scores,Sleep Hours,Sample Question Papers Practiced,Performance Index,Extracurricular Activities_encoded
0,7,99,9,1,91.0,1
1,4,82,4,2,65.0,0
2,8,51,7,2,45.0,1
3,5,52,5,2,36.0,1
4,7,75,8,5,66.0,0
...,...,...,...,...,...,...
9995,1,49,4,2,23.0,1
9996,7,64,8,5,58.0,1
9997,6,83,8,5,74.0,1
9998,9,97,7,0,95.0,1


Создание фрейма данных из свойств данных.
Вывод статистических характеристик исследуемого набора данных

In [51]:
X = student_performance.drop('Performance Index', axis=1)

X.describe()

,Hours Studied,Previous Scores,Sleep Hours,Sample Question Papers Practiced,Extracurricular Activities_encoded
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4.992900,69.445700,6.530600,4.583300,0.494800
std,2.589309,17.343152,1.695863,2.867348,0.499998
min,1.000000,40.000000,4.000000,0.000000,0.000000
25%,3.000000,54.000000,5.000000,2.000000,0.000000
50%,5.000000,69.000000,7.000000,5.000000,0.000000
75%,7.000000,85.000000,8.000000,7.000000,1.000000
max,9.000000,99.000000,9.000000,9.000000,1.000000


Создание фрейма данных для целевой переменной Performance Index

In [52]:
y = pd.DataFrame(student_performance, columns=['Performance Index'])

y.head()

,Performance Index
0,91.0
1,65.0
2,45.0
3,36.0
4,66.0


Признаков не так много, поэтому можно отобрать все признаки, как обладающие наиболее выраженной взаимосвязью с целевой переменной. Создаем список имен признаков, где будут содержаться все признаки.

In [53]:
FEATURE_NAMES = X.columns

X.head()

,Hours Studied,Previous Scores,Sleep Hours,Sample Question Papers Practiced,Extracurricular Activities_encoded
0,7,99,9,1,1
1,4,82,4,2,0
2,8,51,7,2,1
3,5,52,5,2,1
4,7,75,8,5,0


Классы несбалансированы. Баланс классов (Увеличение выборки (Oversampling)). Увеличение выборки заключается в добавлении дополнительных примеров класса-меньшинства, чтобы сделать его более представительным.

In [54]:
ros = RandomOverSampler()

X_resampled, y_resampled = ros.fit_resample(X, y)

Разделяем набора данных в отношении 90/10 с помощью scikit-learn

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.10, random_state=20)

Создаем экземпляр класса LinearRegression, подгоняем модель на тренировочных данных и предсказываем по тестовым данным.
Получаем отчет о множественной регрессии

In [56]:
Linreg = LinearRegression()
Linreg.fit(X_train, y_train.values.ravel())
y_pred = Linreg.predict(X_test)

In [57]:
print(f"root_mean_squared_error: {root_mean_squared_error(y_test, y_pred):.2f}")
print(f"mean_absolute_error:  {mean_absolute_error(y_test, y_pred):.2f}")
print(f"Коэффициент детерминации R^2:  {Linreg.score(X_test, y_test):.2f}")

root_mean_squared_error: 2.01
mean_absolute_error:  1.60
Коэффициент детерминации R^2:  0.99


Использование кросс-валидации для оценки производительности модели, позволяет избежать переобучения и обеспечивая надежность.
Аргумент cv=5 - 5-кратная кросс-валидация. 4 части данных используются для обучения модели, 1 часть - для тестирования.

In [58]:
scores = cross_val_score(Linreg, X_resampled, y_resampled, cv=5)

In [59]:
print(f"Точность на каждой cv: {scores}")
print(f"Средняя точность: {np.mean(scores):.2f}")

Точность на каждой cv: [0.988 0.988 0.99  0.97  0.959]
Средняя точность: 0.98


Вывод по отчету: модель машинного обучения показала хорошие результаты, ошибки минимальны. 
Коэффициент детерминации R^2 - показатель оценки качества регрессивной модели. Он почти равен единице. 
Но использование в качестве оценки только коэффициент детерминации будет ошибочным.
Использовав метод кросс-валидации, был получен примерно тот же результат. Средняя точность примерно равна 1. 
Заметим, что такая точность была достигнута в том числе и благодаря тому, что в качестве важных признаков были выделены все признаки из фрейма данных.